In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import scipy as sp

from src.data import load_riser_data, split_curve

In [240]:
df_riser = load_riser_data('data/riser_large.csv')

df_train, df_test = split_curve(df_riser, 1/4)

df_train = df_train[['bsw', 'rgl', 'qliq', 'psup', 'delta_p']]  # sort by order of breakpoints
df_train = df_train.round(6)

df_train.nunique()

bsw          5
rgl         10
qliq         6
psup         2
delta_p    533
dtype: int64

In [241]:
df_train.apply([np.min, np.max]).T

,amin,amax
bsw,0.010000,0.990000
rgl,50.000000,1000.000000
qliq,300.000000,6500.000000
psup,10.000000,60.000000
delta_p,22.546241,492.470025


In [308]:
X = df_train.values[:,:-1]
y = df_train['delta_p'].values

# only rgl and bsw, for now
X = X[:50]
y = y[:50]

X.shape

(50, 4)

In [328]:
# unique values of x
X_breakpoints = [np.sort(np.unique(x)) for x in X.T]
X_breakpoints = X_breakpoints[:2]  # only rgl and bsw, for now
n_X = list(map(len, X_breakpoints))

# curve = df_train.set_index(['bsw', 'rgl', 'qliq', 'psup',])['delta_p']
curve = df_train[:50].set_index(['bsw', 'rgl',])['delta_p']

In [344]:
# x_input = np.array([0.01, 100, 700, 50])
x_input = np.array([0.02, 100,])

eta = list()
hypercube_limits = list()
for i in range(len(x_input)):
    x_i_breakpoints = X_breakpoints[i]
    
    # faces of the hypercube of breakpoints that enclose the input
    x_i_L = x_i_breakpoints[x_i_breakpoints <= x_input[i]][-1]
    x_i_H = x_i_breakpoints[x_i_breakpoints > x_input[i]][0]
    hypercube_limits.append([x_i_L, x_i_H])

    # relative position of the input with respect to the hypercube's faces
    eta.append((x_input[i] - x_i_L) / (x_i_H - x_i_L))

hypercube_limits = np.array(hypercube_limits)
hypercube_limits

array([[1.00000000e-02, 2.16316000e-01],
       [5.00000000e+01, 1.47435897e+02]])

In [345]:
n = len(n_X)  # number of variables

# weight of each vertex of the hypercube.
theta = np.zeros((2**len(n_X),2))

# indices of theta in crescent order, that is, theta_indices[3] == [0,1,1] means
# that theta[3] is associated with the vertex with x0_L, x1_H and x2_H
theta_indices = np.mgrid[*[slice(2) for _ in range(len(n_X))]].T.reshape(-1,len(n_X))[:,::-1]

# constraints on "high" faces
A_H = list()
b_H = list()
for i in range(n):
    # weights of each theta variable
    a = theta_indices[:,i] == 1
    b_i = eta[i]

    A_H.append(a.astype(float))
    b_H.append(b_i)

A_H = np.stack(A_H)
b_H = np.array(b_H)

# constraints on "low" faces
A_L = list()
b_L = list()
for i in range(n):
    # weights of each theta variable
    a = theta_indices[:,i] == 0
    b_i = 1 - eta[i]

    A_L.append(a.astype(float))
    b_L.append(b_i)

A_L = np.stack(A_L)
b_L = np.array(b_L)

assert np.all(A_L + A_H == 1)
assert np.all(b_L + b_H == 1)

In [346]:
y_vertices = list()
for theta_index in theta_indices:
    x_vertex = np.diag(hypercube_limits.T[theta_index])
    y_vertices.append(curve.loc[*x_vertex])

y_vertices = np.array(y_vertices)

A_y = -y_vertices[None,:]
b_y = np.array([0.,])

A_y, b_y

(array([[ -98.266316,  -38.744453, -101.191013,  -42.463915]]), array([0.]))

In [347]:
A = np.bmat([
    [A_L,np.zeros((len(n_X),1))],
    [A_H,np.zeros((len(n_X),1))],
    [A_y,np.ones((1,1))],
])
b = np.bmat([b_L, b_H, b_y]).T

A.shape, b.shape

((5, 5), (5, 1))

In [348]:
# maximize the y value
c = np.eye(len(theta) + 1)[-1]
c.shape

(5,)

In [349]:
bounds = [(0.,1.),] * len(theta)
bounds.append((0,None))

In [350]:
from scipy.optimize import linprog


res = linprog(c, A_eq=A, b_eq=b, bounds=bounds)

res.x

array([4.38372765e-01, 5.13157897e-01, 4.84693383e-02, 0.00000000e+00,
       6.78639601e+01])

In [351]:
list(zip(res.x, theta_indices))

[(0.438372764670858, array([0, 0])),
 (0.5131578970325483, array([0, 1])),
 (0.04846933829659358, array([1, 0])),
 (0.0, array([1, 1]))]

In [352]:
y_vertices

array([ 98.266316,  38.744453, 101.191013,  42.463915])

In [353]:
eta

[0.04846933829659358, 0.5131578970325484]

### get all combinations with one dimension fixed

In [178]:
n=3 # number of vars
high = False
i=1

theta_indices = np.mgrid[:2,:2,:2].T.reshape(-1,3)[:,::-1]
theta_indices.reshape(-1,2**(n-i-1),3)[int(high)::2]

array([[[0, 0, 0],
        [0, 0, 1]],

       [[1, 0, 0],
        [1, 0, 1]]])

In [191]:
n=3 # number of vars
high = True
i=0

theta_indices = np.array([tis[0]+tis[1]+tis[2] for tis in theta_indices.astype(str)])
theta_indices.reshape(-1,2**(n-i-1))[int(high)::2]

array([['100', '101', '110', '111']], dtype='<U3')